## 1.下载Wikipedia中文语料库
zhwiki-20190720-pages-articles-multistream.xml.bz2

## 2. 通过WikiExtrctor提取语料库
将提取的语料库存到两个文件中（1000M/document）：wiki_00，wiki_01

## 3. 使用opencc将文件中的繁体字转换为简体字
转换的文件wiki_00.txt，wiki_01.txt

In [5]:
pip install opencc

  Stored in directory: /Users/zhangtingshuo/Library/Caches/pip/wheels/a2/fc/85/880732d25bda4ada0c4b4e505483dcfcac117e70511e9156aa
Successfully built opencc
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import opencc

In [2]:
cc = opencc.OpenCC('t2s')
print(cc.convert('Open Chinese Convert（OpenCC）「開放中文轉換」，是一個致力於中文簡繁轉換的項目，提供高質量詞庫和函數庫(libopencc)。'))

Open Chinese Convert（OpenCC）「开放中文转换」，是一个致力于中文简繁转换的项目，提供高质量词库和函数库(libopencc)。


## 4. 正则剔除语料库无效内容
WikiExtractor提取语料库内容储存到文件中时存在较多空行以及</doc...>和<doc\>，通过正则表达式将这一部分内容剔除<br> 再将处理过后的语料库内容进行jieba分词，对后面word2vec模型训练做准备

In [2]:
import re
import jieba

In [41]:
def parse_wiki(input_path, output_path):
    
    remove_part = '[^<doc>.*]|[^</doc>]' #想要移除的部分的正则表达式
    
    input_wiki = open(input_path, 'r', encoding='utf-8')
    
    article = ''
    
    times = 0
    
    with open(output_path, 'w') as o:
        
        for line in input_wiki.readlines():
            
            line = line.strip('\n') # 读取文件时每行会有看不到的\n,去除这部分
            
            if line=='': continue # 如果是空行，continue循环
                
            match_result = re.match(remove_part, line) # 正则匹配match，包含正则表达式的部分将被剔除
            
            if match_result: 
                
                words = jieba.cut(line) # 将不包含无效内容的句子进行jieba分词为之后模型训练准备
                for word in words: article += word + ' '
                    
                o.write(article + '\n') # 将最终结果保存到新文件中
                
                article = ''
                
            times += 1 # 检测输出结果，每一万行输出一下检测程序是否正常进行
            if times%10000 == 0:
                print(times)
                

In [45]:
parse_wiki('/Users/zhangtingshuo/Desktop/AI课程/人工智能与自然语言处理-Part4/zhwiki/AA/wiki_00.txt', '/Users/zhangtingshuo/Desktop/AI课程/人工智能与自然语言处理-Part4/zhwiki/AA/wiki_clean_00.txt')

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390

In [46]:
parse_wiki('/Users/zhangtingshuo/Desktop/AI课程/人工智能与自然语言处理-Part4/zhwiki/AA/wiki_01.txt', '/Users/zhangtingshuo/Desktop/AI课程/人工智能与自然语言处理-Part4/zhwiki/AA/wiki_clean_01.txt')

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390

将两个处理过的语料库文件合并到一块，方便训练模型

In [17]:
def merge(input_path, output_path):
    times = 0
    with open(output_path, 'w') as o:
        for path in input_path:
            input_wiki = open(path, 'r', encoding='utf-8')
            for line in input_wiki:
                line = ''.join(line).replace('，', '').replace('。', '').replace('？', '').replace('！', '') \
                .replace('“', '').replace('”', '').replace('：', '').replace('…', '').replace('（', '').replace('）', '') \
                .replace('—', '').replace('《', '').replace('》', '').replace('、', '').replace('‘', '') \
                .replace('’', '').replace('"', '').replace('.', '').replace('「', '').replace('」', '').replace(';', '') \
                .replace('{', '').replace('}', '').replace('/', '').replace('·', '').replace('；', '').replace('?', '') \
                .replace(',', '').replace('-', '').replace('[', '').replace(']', '').replace('*', '').replace(':', '') \
                .replace('(', '').replace(')', '').replace("'", '').replace('<', '').replace('>', '').replace('『', '') \
                .replace('』', '')
                o.write(line + '\n')
                times += 1
                if times%500000 == 0: print(times)

In [73]:
input_path = ['tttttt.txt', 'test.txt']
merge(input_path, '1234.txt')

In [19]:
input_path = ['/Users/zhangtingshuo/Desktop/AI课程/人工智能与自然语言处理-Part4/zhwiki/AA/wiki_clean_00.txt',
'/Users/zhangtingshuo/Desktop/AI课程/人工智能与自然语言处理-Part4/zhwiki/AA/wiki_clean_01.txt']

output_path='/Users/zhangtingshuo/Desktop/AI课程/人工智能与自然语言处理-Part4/zhwiki/AA/zhwiki_merge.txt'

merge(input_path, output_path)

500000
1000000
1500000
2000000
2500000
3000000
3500000
4000000
4500000
5000000
5500000


## 5. 通过genism库进行模型训练 

In [1]:
pip install gensim

     |████████████████████████████████| 24.7MB 31kB/s eta 0:00:015    |██▉                             | 2.2MB 24kB/s eta 0:15:09     |██████████████████████▊         | 17.6MB 47kB/s eta 0:02:32     |████████████████████████████▍   | 21.9MB 21kB/s eta 0:02:09
     |████████████████████████████████| 71kB 19kB/s eta 0:00:01
     |████████████████████████████████| 133kB 23kB/s eta 0:00:01
     |████████████████████████████████| 71kB 32kB/s eta 0:00:01
     |████████████████████████████████| 5.6MB 33kB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-1.8.4-cp37-none-any.whl size=68202 sha256=ccb17558bfe9bdb7c7d5735aa945355eaebf145c5b6ac7aa2353de97f25209b1
  Stored in directory: /Users/zhangtingshuo/Library/Caches/pip/wheels/5f/ea/fb/5b1a947b369724063b2617011f1540c44eb00e28c3d2ca8692
Successfully built smart-open
Note: you may need to restart the kernel to use updated packages.


In [99]:
from gensim.models import word2vec
import logging

In [138]:
def trainging(save_path):
    
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    
    sentences = word2vec.LineSentence('/Users/zhangtingshuo/Desktop/AI课程/人工智能与自然语言处理-Part4/zhwiki/AA/zhwiki_merge.txt')
    
    model = word2vec.Word2Vec(sentences, size = 250, hs=1)
    
    model.save(save_path)
    
    model.wv.save_word2vec_format('test_wiki' + ".bin", binary=True)

In [139]:
save_path ='/Users/zhangtingshuo/Desktop/AI课程/人工智能与自然语言处理-Part4/zhwiki/AA/zhwiki.model'
trainging(save_path)

2019-08-03 19:25:31,874 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2019-08-03 19:25:31,876 : INFO : collecting all words and their counts
2019-08-03 19:25:31,882 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-08-03 19:25:32,186 : INFO : PROGRESS: at sentence #10000, processed 759376 words, keeping 77414 word types
2019-08-03 19:25:32,456 : INFO : PROGRESS: at sentence #20000, processed 1414372 words, keeping 113939 word types
2019-08-03 19:25:32,723 : INFO : PROGRESS: at sentence #30000, processed 2110546 words, keeping 145584 word types
2019-08-03 19:25:32,960 : INFO : PROGRESS: at sentence #40000, processed 2737604 words, keeping 168679 word types
2019-08-03 19:25:33,265 : INFO : PROGRESS: at sentence #50000, processed 3370095 words, keeping 191808 word types
2019-08-03 19:25:33,487 : INFO : PROGRESS: at sentence #60000, processed 3998304 words, keeping 213874 word types
2019-08-03 19:25:33,809 : INFO : PROGRESS

2019-08-03 19:25:46,599 : INFO : PROGRESS: at sentence #700000, processed 37207614 words, keeping 882224 word types
2019-08-03 19:25:46,811 : INFO : PROGRESS: at sentence #710000, processed 37630879 words, keeping 888866 word types
2019-08-03 19:25:47,047 : INFO : PROGRESS: at sentence #720000, processed 38141572 words, keeping 896360 word types
2019-08-03 19:25:47,228 : INFO : PROGRESS: at sentence #730000, processed 38622799 words, keeping 903580 word types
2019-08-03 19:25:47,411 : INFO : PROGRESS: at sentence #740000, processed 39119294 words, keeping 910693 word types
2019-08-03 19:25:47,602 : INFO : PROGRESS: at sentence #750000, processed 39601216 words, keeping 918412 word types
2019-08-03 19:25:47,825 : INFO : PROGRESS: at sentence #760000, processed 40067753 words, keeping 925525 word types
2019-08-03 19:25:48,069 : INFO : PROGRESS: at sentence #770000, processed 40523010 words, keeping 931798 word types
2019-08-03 19:25:48,258 : INFO : PROGRESS: at sentence #780000, processe

2019-08-03 19:26:00,207 : INFO : PROGRESS: at sentence #1400000, processed 67368018 words, keeping 1296461 word types
2019-08-03 19:26:00,433 : INFO : PROGRESS: at sentence #1410000, processed 67767896 words, keeping 1301527 word types
2019-08-03 19:26:00,628 : INFO : PROGRESS: at sentence #1420000, processed 68154836 words, keeping 1305972 word types
2019-08-03 19:26:00,800 : INFO : PROGRESS: at sentence #1430000, processed 68570616 words, keeping 1311243 word types
2019-08-03 19:26:00,970 : INFO : PROGRESS: at sentence #1440000, processed 68965355 words, keeping 1315882 word types
2019-08-03 19:26:01,153 : INFO : PROGRESS: at sentence #1450000, processed 69364262 words, keeping 1320574 word types
2019-08-03 19:26:01,332 : INFO : PROGRESS: at sentence #1460000, processed 69756899 words, keeping 1324991 word types
2019-08-03 19:26:01,495 : INFO : PROGRESS: at sentence #1470000, processed 70163212 words, keeping 1330017 word types
2019-08-03 19:26:01,671 : INFO : PROGRESS: at sentence #

2019-08-03 19:26:12,680 : INFO : PROGRESS: at sentence #2100000, processed 94038263 words, keeping 1625158 word types
2019-08-03 19:26:12,867 : INFO : PROGRESS: at sentence #2110000, processed 94415846 words, keeping 1629406 word types
2019-08-03 19:26:13,031 : INFO : PROGRESS: at sentence #2120000, processed 94802000 words, keeping 1634013 word types
2019-08-03 19:26:13,207 : INFO : PROGRESS: at sentence #2130000, processed 95195497 words, keeping 1638365 word types
2019-08-03 19:26:13,379 : INFO : PROGRESS: at sentence #2140000, processed 95582665 words, keeping 1642695 word types
2019-08-03 19:26:13,543 : INFO : PROGRESS: at sentence #2150000, processed 95987840 words, keeping 1649455 word types
2019-08-03 19:26:13,705 : INFO : PROGRESS: at sentence #2160000, processed 96386608 words, keeping 1654898 word types
2019-08-03 19:26:13,874 : INFO : PROGRESS: at sentence #2170000, processed 96789267 words, keeping 1659787 word types
2019-08-03 19:26:14,072 : INFO : PROGRESS: at sentence #

2019-08-03 19:26:24,536 : INFO : PROGRESS: at sentence #2790000, processed 119750162 words, keeping 1940299 word types
2019-08-03 19:26:24,603 : INFO : PROGRESS: at sentence #2800000, processed 119879148 words, keeping 1943343 word types
2019-08-03 19:26:24,666 : INFO : PROGRESS: at sentence #2810000, processed 119995330 words, keeping 1945495 word types
2019-08-03 19:26:24,721 : INFO : PROGRESS: at sentence #2820000, processed 120088717 words, keeping 1947695 word types
2019-08-03 19:26:24,809 : INFO : PROGRESS: at sentence #2830000, processed 120286129 words, keeping 1950231 word types
2019-08-03 19:26:24,900 : INFO : PROGRESS: at sentence #2840000, processed 120495532 words, keeping 1953033 word types
2019-08-03 19:26:24,964 : INFO : PROGRESS: at sentence #2850000, processed 120615970 words, keeping 1955479 word types
2019-08-03 19:26:25,040 : INFO : PROGRESS: at sentence #2860000, processed 120771251 words, keeping 1958089 word types
2019-08-03 19:26:25,144 : INFO : PROGRESS: at se

...

2019-08-03 20:19:54,359 : INFO : EPOCH 5 - PROGRESS: at 43.25% examples, 260362 words/s, in_qsize 6, out_qsize 0
2019-08-03 20:19:55,360 : INFO : EPOCH 5 - PROGRESS: at 43.38% examples, 260354 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:19:56,367 : INFO : EPOCH 5 - PROGRESS: at 43.52% examples, 260317 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:19:57,378 : INFO : EPOCH 5 - PROGRESS: at 43.66% examples, 260350 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:19:58,382 : INFO : EPOCH 5 - PROGRESS: at 43.80% examples, 260427 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:19:59,414 : INFO : EPOCH 5 - PROGRESS: at 43.94% examples, 260424 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:20:00,432 : INFO : EPOCH 5 - PROGRESS: at 44.07% examples, 260426 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:20:01,447 : INFO : EPOCH 5 - PROGRESS: at 44.20% examples, 260456 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:20:02,486 : INFO : EPOCH 5 - PROGRESS: at 44.34% examples, 260519 words/s, in_qsiz

2019-08-03 20:21:08,928 : INFO : EPOCH 5 - PROGRESS: at 56.57% examples, 261143 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:21:09,991 : INFO : EPOCH 5 - PROGRESS: at 56.68% examples, 260893 words/s, in_qsize 6, out_qsize 0
2019-08-03 20:21:11,030 : INFO : EPOCH 5 - PROGRESS: at 56.87% examples, 260757 words/s, in_qsize 6, out_qsize 0
2019-08-03 20:21:12,668 : INFO : EPOCH 5 - PROGRESS: at 56.91% examples, 259963 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:21:13,703 : INFO : EPOCH 5 - PROGRESS: at 56.98% examples, 259673 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:21:14,728 : INFO : EPOCH 5 - PROGRESS: at 57.21% examples, 259692 words/s, in_qsize 6, out_qsize 0
2019-08-03 20:21:15,849 : INFO : EPOCH 5 - PROGRESS: at 57.35% examples, 259499 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:21:16,903 : INFO : EPOCH 5 - PROGRESS: at 57.53% examples, 259418 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:21:17,908 : INFO : EPOCH 5 - PROGRESS: at 57.80% examples, 259391 words/s, in_qsiz

2019-08-03 20:22:24,443 : INFO : EPOCH 5 - PROGRESS: at 70.87% examples, 260388 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:22:25,447 : INFO : EPOCH 5 - PROGRESS: at 71.03% examples, 260436 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:22:26,506 : INFO : EPOCH 5 - PROGRESS: at 71.19% examples, 260470 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:22:27,535 : INFO : EPOCH 5 - PROGRESS: at 71.34% examples, 260451 words/s, in_qsize 6, out_qsize 0
2019-08-03 20:22:28,551 : INFO : EPOCH 5 - PROGRESS: at 71.50% examples, 260506 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:22:29,569 : INFO : EPOCH 5 - PROGRESS: at 71.65% examples, 260559 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:22:30,708 : INFO : EPOCH 5 - PROGRESS: at 71.80% examples, 260502 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:22:31,724 : INFO : EPOCH 5 - PROGRESS: at 71.93% examples, 260488 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:22:32,750 : INFO : EPOCH 5 - PROGRESS: at 72.08% examples, 260557 words/s, in_qsiz

2019-08-03 20:23:39,080 : INFO : EPOCH 5 - PROGRESS: at 81.38% examples, 260586 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:23:40,089 : INFO : EPOCH 5 - PROGRESS: at 81.57% examples, 260641 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:23:41,101 : INFO : EPOCH 5 - PROGRESS: at 81.76% examples, 260710 words/s, in_qsize 6, out_qsize 0
2019-08-03 20:23:42,128 : INFO : EPOCH 5 - PROGRESS: at 81.91% examples, 260751 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:23:43,149 : INFO : EPOCH 5 - PROGRESS: at 82.08% examples, 260748 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:23:44,157 : INFO : EPOCH 5 - PROGRESS: at 82.23% examples, 260778 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:23:45,189 : INFO : EPOCH 5 - PROGRESS: at 82.40% examples, 260817 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:23:46,227 : INFO : EPOCH 5 - PROGRESS: at 82.57% examples, 260841 words/s, in_qsize 4, out_qsize 1
2019-08-03 20:23:47,247 : INFO : EPOCH 5 - PROGRESS: at 82.73% examples, 260888 words/s, in_qsiz

2019-08-03 20:24:53,679 : INFO : EPOCH 5 - PROGRESS: at 92.19% examples, 260248 words/s, in_qsize 6, out_qsize 0
2019-08-03 20:24:54,685 : INFO : EPOCH 5 - PROGRESS: at 92.32% examples, 260228 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:24:55,700 : INFO : EPOCH 5 - PROGRESS: at 92.49% examples, 260256 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:24:56,706 : INFO : EPOCH 5 - PROGRESS: at 92.66% examples, 260303 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:24:57,724 : INFO : EPOCH 5 - PROGRESS: at 92.81% examples, 260347 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:24:58,730 : INFO : EPOCH 5 - PROGRESS: at 92.96% examples, 260355 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:24:59,731 : INFO : EPOCH 5 - PROGRESS: at 93.11% examples, 260377 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:25:00,746 : INFO : EPOCH 5 - PROGRESS: at 93.25% examples, 260379 words/s, in_qsize 5, out_qsize 0
2019-08-03 20:25:01,775 : INFO : EPOCH 5 - PROGRESS: at 93.44% examples, 260431 words/s, in_qsiz

In [145]:
model = word2vec.Word2Vec.load('/Users/zhangtingshuo/Desktop/AI课程/人工智能与自然语言处理-Part4/zhwiki/AA/zhwiki.model')
# two_corpus = ['我','哈']
# print(model['我'.decode('utf-8')])
res = model.wv.similarity('人工智能', '神经网络')
print("similarity:%.4f"%res)

2019-08-03 20:47:41,884 : INFO : loading Word2Vec object from /Users/zhangtingshuo/Desktop/AI课程/人工智能与自然语言处理-Part4/zhwiki/AA/zhwiki.model
2019-08-03 20:47:53,902 : INFO : loading wv recursively from /Users/zhangtingshuo/Desktop/AI课程/人工智能与自然语言处理-Part4/zhwiki/AA/zhwiki.model.wv.* with mmap=None
2019-08-03 20:47:53,904 : INFO : loading vectors from /Users/zhangtingshuo/Desktop/AI课程/人工智能与自然语言处理-Part4/zhwiki/AA/zhwiki.model.wv.vectors.npy with mmap=None
2019-08-03 20:47:55,544 : INFO : setting ignored attribute vectors_norm to None
2019-08-03 20:47:55,554 : INFO : loading vocabulary recursively from /Users/zhangtingshuo/Desktop/AI课程/人工智能与自然语言处理-Part4/zhwiki/AA/zhwiki.model.vocabulary.* with mmap=None
2019-08-03 20:47:55,556 : INFO : loading trainables recursively from /Users/zhangtingshuo/Desktop/AI课程/人工智能与自然语言处理-Part4/zhwiki/AA/zhwiki.model.trainables.* with mmap=None
2019-08-03 20:47:55,558 : INFO : loading syn1 from /Users/zhangtingshuo/Desktop/AI课程/人工智能与自然语言处理-Part4/zhwiki/AA/zhwiki.mode

similarity:0.4927
